In [ ]:
from random import random
import numpy as np
import time

In [ ]:
def FW_Q(Q, c, x, verbosity, maxit, maxtime, eps, fstop, stopcr):

    gamma = 1e-4

    flagls = 0

    n = len(Q[0])

    if (maxit < 1000000):
        fh = np.zeros(1, maxit)
        timeVec = np.zeros(1, maxit)
    else:
        fh = np.zeros(1, 100 * n)
        timeVec = np.zeros(1, 100 * n)

    it = 1

    tstart = time.time()

    while it <= maxit and flagls == 0:
    
        Qx = np.dot(Q, x)
        xQx= np.dot(np.transpose(x), Qx)
        cx = np.dot(np.transpose(c), x)
    
        if it == 1:
            fx = 0.5 * xQx - cx
            timeVec[it] = 0
        else:
            #timeVec[it] = cputime - t
            timeVec[it] = time.time()
    
    
        fh[it] = fx
            
        # gradient evaluation
        g = Qx - c
    
        if (timeVec(it) > maxtime):
            break
    
        #solution of FW problem
        istar = np.argmin(g)
        xstar = np.zeros(n,1)
        xstar[istar] = 1.0

        #direction calculation
        d = xstar - x
        gnr = np.dot(np.transpose(g), d)
        
        # stopping criteria and test for termination
        if stopcr == 1:
            if (fx <= fstop):
                    break
        elif stopcr == 2:
            if (gnr >= -eps):
                break
        else:
            raise ValueError('Unknown stopping criterion')
        
    #Armijo search
    alpha = 1
    ref = gamma * gnr

    while 1:
        #z = x + alpha * d
        #Smart computation of the o.f. at the trial point
        fz = 0.5 * ((1-alpha)^2 * xQx + 2 * alpha * (1 - alpha) * Qx[istar] + alpha^2 * Q[istar,istar]) - ((1 - alpha) * cx + alpha * c[istar])

        if fz <= fx + alpha * ref:
            z = x + alpha * d
            break
        else:
            print('alpha trial   = ', alpha, fz, fx)
            alpha = alpha * 0.5

        if alpha <= 1e-20:
            z = x
            fz = fx
            flagls = 1
            it = it - 1
            break

    x = z
    fx = fz

    if verbosity > 0:
        print('-----------------** ', it, ' **------------------')
        print('f(x)     = ', fx)

    it = it + 1

x, it, fx, ttot, fh, timeVec = FW_Q(Q, c, x, verbosity, maxit, maxtime, eps, fstop, stopcr)

#ttot = cputime - t
ttot = time.time()

if it < len(fh[0]):
    fh = fh[1:it]
    timeVec = timeVec[1:it]
    
#x = full(x)

In [ ]:
def Main_MEB():

    #number of samples
    m=2^5
    
    #number of variables
    n=2^12

    # Number of runs for each instance:
    nrun = 1

    # Number of solvers:
    nsolvers = 3

    maxit = 300
    maxtime = 100

    frun_cell = list(nrun, nsolvers)
    timeVectot_cell = list(nrun,nsolvers)

    fstop = np.zeros(nrun,1)

    for krun in range(nrun):

        # Generation of the instance: 

        # seed changes at every run to generate starting point 
        set.seed(krun)

        x0 = np.zeros(n,1)
        x0[1] = 1e0

        Q = random.randn(m,n)
        c = sum(np.power(Q, 2), 1)
        Q = 2e0*(np.dot(np.transpose(Q), Q))

        #--------------------------------------------------------

        count = 1
        eps = 1e-6
        stopcr = 2


        print('****************')
        print('* AWAY STEP FW *')
        print('****************')


        xfwaw, iterfwaw, fxfwaw, tottimefwaw, frun_cell[krun, count], timeVectot_cell[krun, count] = FWAW_Q(Q,c,x0,0,maxit,maxtime,eps,fstop(krun),stopcr)


    # Print results:%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
        print('0.5*xQX - cx = %10.3e\n',fxfwaw)
        print('Number of non-zero components of x = %d\n', sum((abs(xfwaw)>=0.0001)))
        print('Number of iterations = %d\n', iterfwaw)
        print('CPU time = %10.3e\n', tottimefwaw)
        
        #----------------------------------------------------------

        count = count + 1

        print('*****************')
        print('*  FW STANDARD  *')
        print('*****************')


        xfw, iterfw, fxfw, tottimefw, frun_cell[krun, count], timeVectot_cell[krun,count] = FW_Q(Q,c,x0,0,maxit,maxtime,eps,fstop(krun),stopcr)

        # Print results:
        print('0.5*xQX - cx = %10.3e\n',fxfw)
        print('Number of non-zero components of x = %d\n', sum((abs(xfw)>=0.0001)))
        print('Number of iterations = %d\n', iterfw)
        print('CPU time = %10.3e\n', tottimefw)
        ###############################################

  

    fstop = np.min(np.concatenate(fxfwaw, fxfw))
    
    maxit = 0
    for i in range(nsolvers):
        for krun in range(nrun):
            maxit = max(maxit, len(frun_cell[krun, i][0]))

    frun = np.zeros(nrun, nsolvers, maxit)
    timerun = np.zeros(nrun, nsolvers, maxit)
    for i in range(nsolvers):
        for krun in range(nrun):
            niter = len(frun_cell[krun, i][0])
            fx = frun_cell[krun, i][niter]
            t = timeVectot_cell[krun, i][niter]
            frun[krun, i, 1:niter] = frun_cell[krun, i]
            frun[krun, i, niter + 1:maxit] = fx
            timerun[krun, i, 1:niter] = timeVectot_cell[krun, i]
            timerun[krun, i, niter + 1:maxit] = t

    #==========================================================================

    frun_plot = np.zeros(nsolvers,maxit)
    timerun_plot = np.zeros(nsolvers,maxit)
    for i in range(nsolvers):
        for krun in range(nrun):
            frun_plot[i,:] = frun_plot[i,:] + np.transpose(max(0e0,(np.squeeze(frun[krun,i,:]) - fstop(krun))))
            timerun_plot[i,:] = timerun_plot[i,:] + np.transpose(np.squeeze(timerun[krun,i,:]))

    frun_plot = frun_plot / nrun
    timerun_plot = timerun_plot / nrun

    #==========================================================================

    plot_threshold = 1e-20

    # AFW
    plot_fwaw = max(frun_plot[1, 1:maxit], plot_threshold)
    plot_tfwaw = timerun_plot[1, 1:maxit]
   

    #==========================================================================

    # FW 
    plotfw = max(frun_plot[2,1:maxit], plot_threshold)
    plottfw = timerun_plot[2,1:maxit]
    #==========================================================================

   
    #==========================================================================

    # All Solvers
    figure
    semilogy(plot_tfwaw(1,:),plot_fwaw(1,:),'b-')
    hold on
    semilogy(plottfw(1,:),plotfw(1,:),'r-')
    title('FW variants - objective function error')
    legend('AFW','FW')
    xlim([0,30]);  ylim([10^(-10),10^5])
    savefig(strcat('Plot_m_',num2str(m),'_n_',num2str(n),'_all_solvers'))
  #  close(gcf)